In [1]:
#coding=utf-8
import xml.dom.minidom
import re
import os
import sys

In [2]:
def remove(data):
    data = re.sub(r'</?p>', "", data)
    data = re.sub(r'</?tt>', "", data)
    data = re.sub(r'<br/>', "", data)
    data = re.sub(r'\<a.*?\>', "", data)
    data = re.sub(r'</a>', "", data)
    data = re.sub(r'\<div.*?\>', "", data)
    data = re.sub(r'\</div\>', "", data)
    data = re.sub(r'\<pre.*?\>', "", data)
    data = re.sub(r'\</pre\>', "", data)
    data = re.sub(r'\<span.*?\>', "", data)
    data = re.sub(r'\</span\>', "", data)
    data = re.sub(r'\<ul.*?\>', "", data)
    data = re.sub(r'</ul\>', "", data)
    data = re.sub(r'\<table.*?\>', "", data)
    data = re.sub(r'\</table\>', "", data)
    data = re.sub(r'\<td.*?\>', "", data)
    data = re.sub(r'\</td\>', "", data)
    data = re.sub(r'\<th.*?\>', "", data)
    data = re.sub(r'\</th\>', "", data)
    data = re.sub(r'\</?del\>', "", data)
    data = re.sub(r'\</?em\>', "", data)
    data = re.sub(r'\</?h3\>', "", data)
    data = re.sub(r'\</?li\>', "", data)
    data = re.sub(r'</?ol>', "", data)
    data = re.sub(r'</?tr>', "", data)
    data = re.sub(r'</?tbody>', "", data)
    data = re.sub(r'\<img.*?\>', "", data)
    data = re.sub(r'\n', " ", data)
    data = re.sub(r'\&gt\;', ">", data)
    data = re.sub(r'\&lt\;', "<", data)
    data = re.sub(r'\&\#91\;', "[", data)
    data = re.sub(r'\&\#93\;', "]", data)
    data = re.sub(r'\&\#8211\;', "-", data)
    data = re.sub(r'\&amp\;', "&", data)
    data = re.sub(r'\<200c\>', "", data)
    data = re.sub(r'\<200b\>', "", data)
    return data

In [3]:
def readInfoFromXML(parseFile):
    """return dataset, a list of dict data points"""
    dp = {}
    title = parseFile.getElementsByTagName('title')
    for i, ti in enumerate(title):
        if i != 0:
            data = ti.firstChild.data
            data = re.sub(r'\[.*?\]\s', "", data)
            data = remove(data)
            dp['title'] = data
    description = parseFile.getElementsByTagName('description')
    for i, des in enumerate(description):
        if i != 0 and des.firstChild != None:
            data = remove(des.firstChild.data)
            dp['description'] = data.encode('utf-8')
    comments = parseFile.getElementsByTagName('comment')
    for i, com in enumerate(comments):
        data = remove(com.firstChild.data)
        dp['comment'] = data.encode('utf-8')
    type = parseFile.getElementsByTagName('type')
    for i, typ in enumerate(type):
        dp['type'] = typ.firstChild.data.encode('utf-8')
    priority = parseFile.getElementsByTagName('priority')
    for i, pri in enumerate(priority):
        dp['priority'] = pri.firstChild.data.encode('utf-8')
    return dp

In [4]:
def getInfoFromXML(fromDir, path):
    fromFile = os.path.join('%s%s' % (fromDir, path))
    dom = xml.dom.minidom.parse(fromFile)
    parseFile = dom.documentElement
    return readInfoFromXML(parseFile)

In [6]:
def getUsefulInfo(filepath):
    dataset = []
    pathDir =  os.listdir(filepath)
    subName = "SPARK"              # now only deal with the spark issue 
    invalidName = "invalid"
    numFiles = 0
    for allDir in pathDir:
        if subName in allDir:
            if invalidName in allDir:
                continue
            dp = getInfoFromXML(filepath, allDir)
            dataset.append(dp)
            numFiles += 1
    return dataset

filePath = "./data-jira/spark-issues/"
datasets = {'spark':getUsefulInfo(filePath)}
total = len(datasets['spark'])
print('Number of issues:', len(datasets['spark']))
print('Percentage of bugs: %.2f' % (1.0 * sum([1 for dp in datasets['spark'] if dp['type'] == 'Bug']) / len(datasets['spark'])))
#datasets['spark']

('Number of issues:', 4729)
Percentage of bugs: 0.41


In [7]:
last_total = sum([len(datasets['spark'])])
for dp in datasets['spark']:
    if 'description' not in dp:
        datasets['spark'].remove(dp)
total = len(datasets['spark'])  
while last_total != total:
    last_total = sum([len(datasets['spark'])])
    for dp in datasets['spark']:
        if 'description' not in dp:
            datasets['spark'].remove(dp)
    total = len(datasets['spark'])
print('Number of issues left: ', total) 

('Number of issues left: ', 4173)


In [8]:
last_total = sum([len(datasets['spark'])])
for dp in datasets['spark']:
    if 'comment' not in dp:
        datasets['spark'].remove(dp)
total = len(datasets['spark'])  
while last_total != total:
    last_total = sum([len(datasets['spark'])])
    for dp in datasets['spark']:
        if 'comment' not in dp:
            datasets['spark'].remove(dp)
    total = len(datasets['spark'])
print('Number of issues left: ', total) 

('Number of issues left: ', 4109)


In [10]:
import pickle
pickle.dump(datasets, open('data-jira/spark-issues.pickle', 'wb'), True)